In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
from snowProfile import SnowProfile, SurfaceCondition, Layer, TempMeasurement

## Try other parser

In [3]:
import xml.etree.ElementTree as ET
file_path = "snowpits_200_MT/snowpits-66387-caaml.xml"

common_tag = '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}'
root = ET.parse(file_path).getroot()


In [4]:
# measurementDirection
try:
    measurementDirection = next(root.iter(common_tag + 'SnowProfileMeasurements'), None).get('dir')
except AttributeError:
    measurementDirection = None

print(measurementDirection)


top down


In [5]:
hs_tag = common_tag + 'profileDepth'
        
try:
    hs = next(root.iter(hs_tag), None).text
    hs_units = next(root.iter(hs_tag), None).get('uom')
    HS = [float(hs), hs_units]
except AttributeError:
    HS = None


Surface Conditions
windLoading
penetrationFoot
penetrationSki

In [6]:
custom = root.iter(common_tag+'custom_Data')

for tag in custom:
    print(tag.text)

In [7]:
try:
    penFoot_val = next(root.iter(common_tag + 'penetrationFoot'), None).text
    penFoot_units = next(root.iter(common_tag + 'penetrationFoot'), None).get('uom')
    penFoot = [float(penFoot_val), penFoot_units]
except AttributeError:
    penFoot = None

In [8]:
# layers
layers = root.iter(common_tag + 'Layer')

layer_list = []

for layer in layers:
    layer_obj = Layer()
    for prop in layer.iter():
        if prop.tag.endswith('depthTop'):
            depthTop_val = prop.text
            depthTop_units = prop.get('uom')
            depthTop=[depthTop_val,depthTop_units]
            layer_obj.set_depthTop(depthTop)

        if prop.tag.endswith('thickness'):
            thickness_val = prop.text
            thickness_units = prop.get('uom')
            thickness=[thickness_val,thickness_units]
            layer_obj   .set_thickness(thickness)
        if prop.tag.endswith('grainFormPrimary'):
            grainFormPrimary = prop.text
            layer_obj.set_grainFormPrimary(grainFormPrimary)
        if prop.tag.endswith('hardness'):
            hardness = prop.text
            layer_obj.set_hardness(hardness)
        if prop.tag.endswith('wetness'):
            wetness = prop.text
            layer_obj.set_wetness(wetness)
        if prop.tag.endswith('layerOfConcern'):
            layerOfConcern = prop.text
            layer_obj.set_layerOfConcern(layerOfConcern)
            

    layer_list.append(layer_obj)

for i,layer in enumerate(layer_list):
    print(f"Layer {i+1}:")

    print(layer_list[i])

        

Layer 1:

	 depthTop: ['0', 'cm']
	 thickness: ['8', 'cm']
	 grainFormPrimary: DFbk
	 hardness: F
	 wetness: None
	 layerOfConcern: None
	 grainFormPrimary_Class: DF
Layer 2:

	 depthTop: ['8', 'cm']
	 thickness: ['39', 'cm']
	 grainFormPrimary: DFbk
	 hardness: F+
	 wetness: None
	 layerOfConcern: None
	 grainFormPrimary_Class: DF
Layer 3:

	 depthTop: ['47', 'cm']
	 thickness: ['1', 'cm']
	 grainFormPrimary: MFcr
	 hardness: 1F
	 wetness: None
	 layerOfConcern: true
	 grainFormPrimary_Class: MF
Layer 4:

	 depthTop: ['48', 'cm']
	 thickness: ['20', 'cm']
	 grainFormPrimary: FCxr
	 hardness: 4F
	 wetness: None
	 layerOfConcern: None
	 grainFormPrimary_Class: FC
Layer 5:

	 depthTop: ['23', 'cm']
	 thickness: None
	 grainFormPrimary: None
	 hardness: None
	 wetness: None
	 layerOfConcern: None
	 grainFormPrimary_Class: None


In [9]:
# pitID
#  <caaml:locRef gml:id="location-nid-66387">

pitID_tag = common_tag + 'locRef'
gml_tag = '{http://www.opengis.net/gml}id'

try:
    pitID_str = next(root.iter(pitID_tag), None).attrib[gml_tag]
    pitID = pitID_str.split('-')[-1]
except AttributeError:
    pitID = None

print(pitID)

66387


In [35]:
#user = {
 #           'OperationID': None,
  #          'OperationName': None,
   #         'Professional': None,
    #        'ContactPersonID': None,
     #       'ContactPersonName': None,
      #      }   

user = {}


try:
    srcRef = root.iter(common_tag + 'srcRef')
except AttributeError:
    srcRef = None

print(srcRef)

if srcRef is not None:
    for prop in srcRef:
        print("tag: ",prop.tag)
        print("attrib: ",prop.attrib)
        print("text: ",prop.text)
        for sub_prop in prop:
            print("sub_tag: ",sub_prop.tag)
            print("sub_attrib: ",sub_prop.attrib)
            print("sub_text: ",sub_prop.text)
            for sub_sub_prop in sub_prop:
                print("sub_sub_tag: ",sub_sub_prop.tag)
                print("sub_sub_attrib: ",sub_sub_prop.attrib)
                print("sub_sub_text: ",sub_sub_prop.text)
                for sub_sub_sub_prop in sub_sub_prop:
                    print("sub_sub_sub_tag: ",sub_sub_sub_prop.tag)
                    print("sub_sub_sub_attrib: ",sub_sub_sub_prop.attrib)
                    print("sub_sub_sub_text: ",sub_sub_sub_prop.text)

if srcRef is not None:
    for prop in srcRef:
        for sub_prop in prop:
            if sub_prop.tag.endswith('Operation'):
                user['OperationID'] = sub_prop.attrib['gml:id']
                for sub_sub_prop in sub_prop:
                    if sub_sub_prop.tag.endswith('name'):
                        user['OperationName'] = sub_sub_prop.text
                    if sub_sub_prop.tag.endswith('contactPerson'):
                        user['ContactPersonID'] = sub_sub_prop.attrib['gml:id']
                        for sub_sub_sub_prop in sub_sub_prop:
                            if sub_sub_sub_prop.tag.endswith('name'):
                                user['ContactPersonName'] = sub_sub_sub_prop.text
            elif sub_prop.tag.endswith('Person'):
                user['ContactPersonID'] = sub_prop.attrib['gml:id']
                for sub_sub_prop in sub_prop:
                    if sub_sub_prop.tag.endswith('name'):
                        user['ContactPersonName'] = sub_sub_prop.text



print(user)


tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}srcRef
attrib:  {}
text:  
    
sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}Operation
sub_attrib:  {'{http://www.opengis.net/gml}id': 'SnowPilot-Group-68'}
sub_text:  
      
sub_sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}name
sub_sub_attrib:  {}
sub_sub_text:  Bridger Bowl Ski Patrol
sub_sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}contactPerson
sub_sub_attrib:  {'{http://www.opengis.net/gml}id': 'SnowPilot-User-2900'}
sub_sub_text:  
        
sub_sub_sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}name
sub_sub_sub_attrib:  {}
sub_sub_sub_text:  BB Ski Patrol
{}
